In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as func



In [2]:
sc = SparkContext("local", "HealthCareCapstone")
sqlContext = SQLContext(sc)

                       

In [3]:

# PART D: https://data.cms.gov/Medicare-Part-D/Medicare-Provider-Utilization-and-Payment-Data-201/mhdd-npjx
# CMS Payments: https://download.cms.gov/openpayments/PGYR19_P012221.ZIP
# Exclusions: https://oig.hhs.gov/exclusions/downloadables/UPDATED.csv
# Opioid Prescribers: https://data.cms.gov/Medicare-Part-D/Medicare-Part-D-Opioid-Prescriber-Summary-File-201/j2ra-95gh
# Opioid List: https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Downloads/OpioidDrugList.zip

#partD_drug_rawdata = sqlContext.read.csv#("D:\\NematiLab\\CMS-Medicare-Data-FRAUD-Detection\\Medicare_Provider_Utilization_and_Payment_Data__2018_Part_D_Prescriber.csv", \
#                                       header = True, inferSchema = True)

#payment_rawdata = sqlContext.read.csv("D:\\NematiLab\\CMS-Medicare-Data-FRAUD-Detection\\PGYR19_P012221\\OP_DTL_GNRL_PGYR2019_P01222021.csv", \
#                                       header = True, inferSchema = True)


#IELE_rawdata = sqlContext.read.csv("D:\\NematiLab\\CMS-Medicare-Data-FRAUD-Detection\\UPDATED.csv", \
#                                       header = True, inferSchema = True)

#Opioids_drugs = sqlContext.read.csv("D:\\NematiLab\\CMS-Medicare-Data-FRAUD-Detection\\Opioid_Drug_Lists_CY2018_0\\Opioid_Drug_Lists_CY2018.xlsx", \
#                                       header = True, inferSchema = True)

Opioids_Prescriber = sqlContext.read.csv("D:\\NematiLab\\CMS-Medicare-Data-FRAUD-Detection\\Medicare_Part_D_Opioid_Prescriber_Summary_File_2018.csv", \
                                       header = True, inferSchema = True)

In [33]:
# load the dataset
#  1. Part D drug 2016 dataset 
#     https://data.cms.gov/Medicare-Part-D/Medicare-Provider-Utilization-and-Payment-Data-201/yvpj-pmj2
#  2. CMS Payment Data Set  
#     https://www.cms.gov/OpenPayments/Explore-the-Data/Dataset-Downloads.html#
#  3. EXCLUEDED Datavset
#     https://oig.hhs.gov/exclusions/exclusions_list.asp 
#  4. Opioids drugs and opioids prescriber information

# partD_drug_rawdata = sc.textFile("/Users/my_macbook/Projects/Project_Capstone/DataSet/PartD_Drug_16/PartD_Drug_16.txt")
# #partD_pres_rawdata = sc.textFile("/Users/my_macbook/Projects/Project_Capstone/DataSet/PartD_Prescriber_16/PartD_Prescriber_16.txt")
# payment_rawdata = sqlContext.read.csv("/Capstone/DataSet/2017Payment/OP_DTL_GNRL_PGYR2017_P06292018.csv", \
#                                        header = True, inferSchema = True)
# IELE_rawdata = sqlContext.read.csv("/Users/my_macbook/Projects/Project_Capstone/DataSet/2018UpdatedLEIE.csv", \
#                                        header = True, inferSchema = True)
# Opioids_drugs = sqlContext.read.csv("/Users/my_macbook/Projects/Project_Capstone/DataSet/opioids.csv", \
#                                        header = True, inferSchema = True)
# Opioids_Prescriber = sqlContext.read.csv("/Users/my_macbook/Projects/Project_Capstone/DataSet/opioids_prescriber_info.csv", \
#                                        header = True, inferSchema = True)




In [10]:
# PAYMENT DATA PREPARATION
payment_features_df = payment_rawdata.select(["Physician_First_Name",\
                                             "Physician_Last_Name", \
                                             "Recipient_City", \
                                             "Recipient_State", \
                                             "Total_Amount_of_Payment_USDollars"])
#payment_features_df.show()



In [11]:
payment_features_df = payment_features_df.withColumn("Total_Amount_of_Payment_USDollars",payment_features_df["Total_Amount_of_Payment_USDollars"].cast(DoubleType()))

payment_features_gourpbyTotal_df = payment_features_df.groupBy("Physician_First_Name","Physician_Last_Name","Recipient_City","Recipient_State").sum("Total_Amount_of_Payment_USDollars")



In [12]:
payment_features_gourpbyTotal_df.show()

+--------------------+-------------------+--------------+---------------+--------------------------------------+
|Physician_First_Name|Physician_Last_Name|Recipient_City|Recipient_State|sum(Total_Amount_of_Payment_USDollars)|
+--------------------+-------------------+--------------+---------------+--------------------------------------+
|              Darryl|             Werner|     Santa Ana|             CA|                               6085.23|
|             Satoshi|          Tateshima|   Los Angeles|             CA|                                502.34|
|             CHARLES|              MOSES|   PORT ARTHUR|             TX|                    29.009999999999998|
|           HAMIDREZA|         IRANMANESH|      KINGWOOD|             TX|                    133.04999999999998|
|                MARC|             LAVINE|  PHILADELPHIA|             PA|                                516.78|
|                ASRA|                ALI|      PEARLAND|             TX|                    150

In [13]:
payment_features_gourpbyTotal_df = payment_features_gourpbyTotal_df.withColumnRenamed("Physician_First_Name" , "first_name")
payment_features_gourpbyTotal_df = payment_features_gourpbyTotal_df.withColumnRenamed("Physician_Last_Name" , "last_name")
payment_features_gourpbyTotal_df = payment_features_gourpbyTotal_df.withColumnRenamed("Recipient_City" , "city")
payment_features_gourpbyTotal_df = payment_features_gourpbyTotal_df.withColumnRenamed("Recipient_State" , "state")
payment_features_gourpbyTotal_df = payment_features_gourpbyTotal_df.withColumnRenamed("sum(Total_Amount_of_Payment_USDollars)" , "Total_Amount_of_Payment")

payment_features_gourpbyTotal_df.show()

+----------+-----------+------------+-----+-----------------------+
|first_name|  last_name|        city|state|Total_Amount_of_Payment|
+----------+-----------+------------+-----+-----------------------+
|    Darryl|     Werner|   Santa Ana|   CA|                6085.23|
|   Satoshi|  Tateshima| Los Angeles|   CA|                 502.34|
|   CHARLES|      MOSES| PORT ARTHUR|   TX|     29.009999999999998|
| HAMIDREZA| IRANMANESH|    KINGWOOD|   TX|     133.04999999999998|
|      MARC|     LAVINE|PHILADELPHIA|   PA|                 516.78|
|      ASRA|        ALI|    PEARLAND|   TX|     1504.7499999999998|
|    ANGELA|SALDARRIAGA|  WILMINGTON|   DE|     262.03000000000003|
|   ZACHARY|    COMPTON|      IRVING|   TX|      758.3100000000001|
|   ELLIOTT|     SILBAR|   MILWAUKEE|   WI|                1131.42|
|    CELINE|    GISBERT|   PENSACOLA|   FL|                 678.64|
|   JANGWON|       YOON|PHILADELPHIA|   PA|                4021.86|
|     ANITA|       IYER|    BETHESDA|   MD|     

In [14]:
payment_features_gourpbyTotal_df = payment_features_gourpbyTotal_df.sort(payment_features_gourpbyTotal_df.Total_Amount_of_Payment.desc())


In [15]:
payment_features_gourpbyTotal_df.show()

+----------+----------+-------------+-----+-----------------------+
|first_name| last_name|         city|state|Total_Amount_of_Payment|
+----------+----------+-------------+-----+-----------------------+
|      null|      null|       DUARTE|   CA|    3.779466413300001E8|
|      null|      null|      Houston|   TX|         3.0784438065E8|
|      null|      null|       BOSTON|   MA|   2.5903488950999996E8|
|    ANDREW|    COOPER|   CLEARWATER|   FL|          5.000821586E7|
|      null|      null|       Boston|   MA|          4.438468705E7|
|    STEVEN|   BOLLING|    ANN ARBOR|   MI|   3.7347662900000006E7|
|   STEPHEN|  BURKHART|  SAN ANTONIO|   TX|          2.716575634E7|
|      null|      null|  Los Angeles|   CA|   2.5334865490000006E7|
|   WILLIAM|    BINDER|BEVERLY HILLS|   CA|          2.072729672E7|
|     KEVIN|     FOLEY|      Memphis|   TN|   2.0217118830000002E7|
|      null|      null|      HOUSTON|   TX|   1.4820399579999998E7|
|      null|      null|        Phila|   PA|   1.

In [20]:
# Part D drug data Preparation 

#header = partD_drug_rawdata.first()
#partD_drug_rawdata =partD_drug_rawdata.filter (lambda line: line != header)

# temp_var = partD_drug_rawdata.map(lambda k: k.split("\t"))
partD_Drug_df = partD_drug_rawdata
partD_Drug_df.show()


partD_Drug_df = partD_Drug_df.withColumn("bene_count",partD_Drug_df["bene_count"].cast(DoubleType()))
partD_Drug_df = partD_Drug_df.withColumn("total_claim_count",partD_Drug_df["total_claim_count"].cast(DoubleType()))
partD_Drug_df = partD_Drug_df.withColumn("total_30_day_fill_count",partD_Drug_df["total_30_day_fill_count"].cast(DoubleType()))
partD_Drug_df = partD_Drug_df.withColumn("total_day_supply",partD_Drug_df["total_day_supply"].cast(DoubleType()))
partD_Drug_df = partD_Drug_df.withColumn("total_drug_cost",partD_Drug_df["total_drug_cost"].cast(DoubleType()))
partD_Drug_df = partD_Drug_df.withColumn("bene_count_ge65",partD_Drug_df["bene_count_ge65"].cast(DoubleType()))
partD_Drug_df = partD_Drug_df.withColumn("total_claim_count_ge65",partD_Drug_df["total_claim_count_ge65"].cast(DoubleType()))
partD_Drug_df = partD_Drug_df.withColumn("total_30_day_fill_count_ge65",partD_Drug_df["total_30_day_fill_count_ge65"].cast(DoubleType()))
partD_Drug_df = partD_Drug_df.withColumn("total_day_supply_ge65",partD_Drug_df["total_day_supply_ge65"].cast(DoubleType()))
partD_Drug_df = partD_Drug_df.withColumn("total_drug_cost_ge65",partD_Drug_df["total_drug_cost_ge65"].cast(DoubleType()))








+----------+----------------------------+-------------------------+-------------------+--------------------+---------------------+----------------+--------------------+--------------------+----------+-----------------+-----------------------+----------------+---------------+---------------+-----------------------------+----------------------+------------------+----------------------------+---------------------+--------------------+
|       npi|nppes_provider_last_org_name|nppes_provider_first_name|nppes_provider_city|nppes_provider_state|specialty_description|description_flag|           drug_name|        generic_name|bene_count|total_claim_count|total_30_day_fill_count|total_day_supply|total_drug_cost|bene_count_ge65|bene_count_ge65_suppress_flag|total_claim_count_ge65|ge65_suppress_flag|total_30_day_fill_count_ge65|total_day_supply_ge65|total_drug_cost_ge65|
+----------+----------------------------+-------------------------+-------------------+--------------------+--------------------

In [21]:
partD_Drug_df.head()

Row(npi=1003000126, nppes_provider_last_org_name='ENKESHAFI', nppes_provider_first_name='ARDALAN', nppes_provider_city='CUMBERLAND', nppes_provider_state='MD', specialty_description='Internal Medicine', description_flag='S', drug_name='ATORVASTATIN CALCIUM', generic_name='ATORVASTATIN CALCIUM', bene_count=None, total_claim_count=13.0, total_30_day_fill_count=13.0, total_day_supply=390.0, total_drug_cost=85.83, bene_count_ge65=None, bene_count_ge65_suppress_flag='*', total_claim_count_ge65=None, ge65_suppress_flag='*', total_30_day_fill_count_ge65=None, total_day_supply_ge65=None, total_drug_cost_ge65=None)

In [22]:
partD_Drug_df.printSchema()

root
 |-- npi: integer (nullable = true)
 |-- nppes_provider_last_org_name: string (nullable = true)
 |-- nppes_provider_first_name: string (nullable = true)
 |-- nppes_provider_city: string (nullable = true)
 |-- nppes_provider_state: string (nullable = true)
 |-- specialty_description: string (nullable = true)
 |-- description_flag: string (nullable = true)
 |-- drug_name: string (nullable = true)
 |-- generic_name: string (nullable = true)
 |-- bene_count: double (nullable = true)
 |-- total_claim_count: double (nullable = true)
 |-- total_30_day_fill_count: double (nullable = true)
 |-- total_day_supply: double (nullable = true)
 |-- total_drug_cost: double (nullable = true)
 |-- bene_count_ge65: double (nullable = true)
 |-- bene_count_ge65_suppress_flag: string (nullable = true)
 |-- total_claim_count_ge65: double (nullable = true)
 |-- ge65_suppress_flag: string (nullable = true)
 |-- total_30_day_fill_count_ge65: double (nullable = true)
 |-- total_day_supply_ge65: double (null

In [23]:
partD_Drug_features_df = partD_Drug_df.select(["npi",\
                                               "nppes_provider_city",\
                                               "nppes_provider_state", \
                                               "nppes_provider_last_org_name", \
                                               "nppes_provider_first_name", \
                                               "specialty_description",\
                                               "drug_name", \
                                               "generic_name",\
                                               "total_drug_cost",\
                                               "total_claim_count",\
                                               "total_day_supply"])

partD_Drug_featuresGroupbyNPI_df = partD_Drug_features_df.groupBy("npi",\
                                                                 "nppes_provider_city",\
                                                                 "nppes_provider_state", \
                                                                 "nppes_provider_last_org_name", \
                                                                 "nppes_provider_first_name", \
                                                                 "specialty_description",\
                                                                 "drug_name", \
                                                                 "generic_name")\
                                                                         .agg(func.sum("total_drug_cost"),\
                                                                              func.sum("total_claim_count"),\
                                                                              func.sum("total_day_supply"), \
                                                                              func.variance("total_drug_cost"),\
                                                                              func.variance("total_claim_count"),\
                                                                              func.variance("total_day_supply"),\
                                                                              func.max("total_drug_cost"),\
                                                                              func.max("total_claim_count"),\
                                                                              func.max("total_day_supply"),\
                                                                             )


partD_Drug_featuresGroupbyNPI_df = partD_Drug_featuresGroupbyNPI_df.withColumnRenamed("nppes_provider_first_name" , "first_name")
partD_Drug_featuresGroupbyNPI_df = partD_Drug_featuresGroupbyNPI_df.withColumnRenamed("nppes_provider_last_org_name" , "last_name")
partD_Drug_featuresGroupbyNPI_df = partD_Drug_featuresGroupbyNPI_df.withColumnRenamed("nppes_provider_city" , "city")
partD_Drug_featuresGroupbyNPI_df = partD_Drug_featuresGroupbyNPI_df.withColumnRenamed("nppes_provider_state" , "state")

In [24]:
partD_Drug_featuresGroupbyNPI_df.show()

+----------+---------------+-----+----------+-----------+---------------------+--------------------+--------------------+--------------------+----------------------+---------------------+-------------------------+---------------------------+--------------------------+--------------------+----------------------+---------------------+
|       npi|           city|state| last_name| first_name|specialty_description|           drug_name|        generic_name|sum(total_drug_cost)|sum(total_claim_count)|sum(total_day_supply)|var_samp(total_drug_cost)|var_samp(total_claim_count)|var_samp(total_day_supply)|max(total_drug_cost)|max(total_claim_count)|max(total_day_supply)|
+----------+---------------+-----+----------+-----------+---------------------+--------------------+--------------------+--------------------+----------------------+---------------------+-------------------------+---------------------------+--------------------------+--------------------+----------------------+------------------

In [ ]:
# EXCLUDED Data Preparation

In [26]:
IELE_rawdata.show()

+--------+---------+-------+--------------------+------------------+-----------------+----+----------+----+--------------------+---------+-----+-----+--------+--------+--------+----------+--------+
|LASTNAME|FIRSTNAME|MIDNAME|             BUSNAME|           GENERAL|        SPECIALTY|UPIN|       NPI| DOB|             ADDRESS|     CITY|STATE|  ZIP|EXCLTYPE|EXCLDATE|REINDATE|WAIVERDATE|WVRSTATE|
+--------+---------+-------+--------------------+------------------+-----------------+----+----------+----+--------------------+---------+-----+-----+--------+--------+--------+----------+--------+
|    null|     null|       |#1 MARKETING SERV...|    OTHER BUSINESS|       SOBER HOME|null|         0|null|239 BRIGHTON BEAC...| BROOKLYN|   NY|11235|  1128a1|20200319|       0|         0|    null|
|    null|     null|       |14 LAWRENCE AVE P...|          PHARMACY|             null|null|         0|null|  14 LAWRENCE AVENUE|SMITHTOWN|   NY|11787|  1128a1|19880830|       0|         0|    null|
|    null|

In [27]:

npi_fraud_df = IELE_rawdata.select (["npi", "EXCLTYPE"])

In [28]:
npi_fraud_df = npi_fraud_df.filter(npi_fraud_df.npi != 0)

In [29]:
npi_fraud_df.printSchema

<bound method DataFrame.printSchema of DataFrame[npi: int, EXCLTYPE: string]>

In [30]:
npi_fraud_df = npi_fraud_df.withColumn("is_fraud", npi_fraud_df["npi"]/npi_fraud_df["npi"] )

In [31]:
npi_fraud_df = npi_fraud_df.withColumn("is_fraud",npi_fraud_df["is_fraud"].cast(IntegerType()))

TypeError: 'Column' object is not callable

In [ ]:
# Opioid drugs

In [34]:
Opioids_Prescriber.printSchema()

root
 |-- NPI: integer (nullable = true)
 |-- NPPES Provider Last Name: string (nullable = true)
 |-- NPPES Provider First Name: string (nullable = true)
 |-- NPPES Provider ZIP Code: integer (nullable = true)
 |-- NPPES Provider State: string (nullable = true)
 |-- Specialty Description: string (nullable = true)
 |-- Total Claim Count: integer (nullable = true)
 |-- Opioid Claim Count: integer (nullable = true)
 |-- Opioid Prescribing Rate: double (nullable = true)
 |-- Long-Acting Opioid Claim Count: integer (nullable = true)
 |-- Long-Acting Opioid Prescribing Rate: double (nullable = true)



In [36]:
# Opioids_Prescriber_npi_df = Opioids_Prescriber.select (["npi","Specialty","is_Opioid_Prescriber"])

Opioids_Prescriber_npi_df = Opioids_Prescriber.select (["npi","Specialty Description"])

In [37]:
Opioids_Prescriber_npi_df.show()

+----------+---------------------+
|       npi|Specialty Description|
+----------+---------------------+
|1003000126|    Internal Medicine|
|1003000142|       Anesthesiology|
|1003000167|              Dentist|
|1003000282|   Nurse Practitioner|
|1003000407|      Family Practice|
|1003000423| Obstetrics & Gyne...|
|1003000480|      General Surgery|
|1003000522|      Family Practice|
|1003000530|    Internal Medicine|
|1003000597|              Urology|
|1003000639|      Cardiac Surgery|
|1003000720|   Nurse Practitioner|
|1003000746|           Pharmacist|
|1003000837|   Nurse Practitioner|
|1003000902|      Family Practice|
|1003000936|    Internal Medicine|
|1003001017|          Dermatology|
|1003001132|      Family Practice|
|1003001207|              Dentist|
|1003001256|      Family Practice|
+----------+---------------------+
only showing top 20 rows



In [38]:
df =  npi_fraud_df.join(Opioids_Prescriber_npi_df, (Opioids_Prescriber_npi_df.npi == npi_fraud_df.npi))

In [39]:
df.printSchema()

root
 |-- npi: integer (nullable = true)
 |-- EXCLTYPE: string (nullable = true)
 |-- is_fraud: integer (nullable = true)
 |-- npi: integer (nullable = true)
 |-- Specialty Description: string (nullable = true)



In [40]:

sqlContext.registerDataFrameAsTable(payment_features_gourpbyTotal_df,"payment_table")
payment_df = sqlContext.sql("Select * from payment_table")


In [41]:
sqlContext.registerDataFrameAsTable(partD_Drug_featuresGroupbyNPI_df,"partD_table")
partD_df = sqlContext.sql("Select * from partD_table")

In [42]:
sqlContext.registerDataFrameAsTable(npi_fraud_df,"npi_fraud_table")
npifraud_df = sqlContext.sql("Select * from npi_fraud_table")

In [43]:
sqlContext.registerDataFrameAsTable(Opioids_Prescriber_npi_df,"npi_opioids_table")
npiopioids_df = sqlContext.sql("Select * from npi_opioids_table")

In [44]:
features_df = sqlContext.sql ("select npi ")


AnalysisException: cannot resolve '`npi`' given input columns: []; line 1 pos 7;
'Project ['npi]
+- OneRowRelation


In [ ]:
SELECT npi
FROM (
    SELECT distinct npi , "first_name" , "last_name", "city" ,"state"
    FROM payment_table
    ) npis 
    
LEFT JOIN  payments pay
ON npis."NPPES_PROVIDER_FIRST_NAME" = pay."Physician_First_Name"
AND npis."NPPES_PROVIDER_LAST_ORG_NAME" = pay."Physician_Last_Name"
AND npis."NPPES_PROVIDER_CITY" = pay."Recipient_City"
AND npis."NPPES_PROVIDER_STATE" = pay."Recipient_State"

GROUP BY npi
;